In [24]:
import pandas as pd
from transformers import AutoModel, AutoTokenizer

In [25]:
model_name = "dkleczek/bert-base-polish-uncased-v1"
model = AutoModel.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of the model checkpoint at dkleczek/bert-base-polish-uncased-v1 were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [26]:
df = pd.read_json("../data/all_cleaned_data.json")
df.head()

,id,text,label
0,1357999284721525,4 000 6 000 zł za spisanie i zamknięcie 1 umo...,NaN
1,1512648315923287,warszawa potrzebuję hostessy lub hosta 30 11 0...,NaN
2,1512730425915076,praca od jutra zatrudnię pełnoletnie osoby do ...,NaN
3,1512510969270355,w związku z dynamicznie rozwijającym się zespo...,NaN
4,1505264493328336,przy komputerze umowa wynagrodzenie co miesiąc...,NaN


In [27]:
def get_embeddings(text):
    encoded_input = tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
    output = model(**encoded_input)
    embeddings = output.last_hidden_state[:,0,:].detach().numpy()[0]
    return embeddings

In [28]:
df['embeddings'] = df['text'].apply(get_embeddings)

In [29]:
df.head()

,id,text,label,embeddings
0,1357999284721525,4 000 6 000 zł za spisanie i zamknięcie 1 umo...,NaN,"[-0.7777841, -0.26344857, 0.68107945, 1.984628..."
1,1512648315923287,warszawa potrzebuję hostessy lub hosta 30 11 0...,NaN,"[-0.8700619, -0.41925764, 0.82610565, 2.029324..."
2,1512730425915076,praca od jutra zatrudnię pełnoletnie osoby do ...,NaN,"[-1.3931414, -0.3115666, 0.7041424, 1.5579505,..."
3,1512510969270355,w związku z dynamicznie rozwijającym się zespo...,NaN,"[-0.7349925, -0.98008686, 0.6741656, 0.9993763..."
4,1505264493328336,przy komputerze umowa wynagrodzenie co miesiąc...,NaN,"[-0.6780882, -0.75793606, 0.411735, 0.90742075..."


In [30]:
df['embeddings'][0].shape

(768,)

In [31]:
df.to_csv('../data/all_data_polbert_embeddings.csv', index=False)
with open('../data/all_data_polbert_embeddings.json', 'w', encoding='utf-8') as f:
  df.to_json(f, force_ascii=False, orient='records')